In [3]:
import torch
import numpy as np
import os
import cv2
from facenet_pytorch import InceptionResnetV1, MTCNN

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load FaceNet model
model = InceptionResnetV1(pretrained="vggface2").to(device).eval()

# Initialize MTCNN face detector
mtcnn = MTCNN(keep_all=False, device=device)

# Set dataset path
dataset_path = r"C:\Code\Alan Guedes - CSMPR21_data\test_reference_images"

# Dictionary to store embeddings
face_embeddings = {}

# Iterate through test_reference_images directory
for file_name in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file_name)

    # Read image
    img = cv2.imread(file_path)
    if img is None:
        print(f"Warning: Could not read {file_name}. Skipping...")
        continue

    # Convert color format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect face
    face = mtcnn(img)
    if face is None:
        print(f"Warning: No face detected in {file_name}. Skipping...")
        continue

    # Extract embedding
    with torch.no_grad():
        embedding = model(face.unsqueeze(0).to(device)).cpu().numpy().flatten()

    # Store in dictionary
    person_id = os.path.splitext(file_name)[0]  # Use filename as ID
    face_embeddings[person_id] = embedding

# Save embeddings to .npy file
embeddings_path = r"C:\Code\vision-attendance-system\face_embeddings.npy"
np.save(embeddings_path, face_embeddings)
print(f"Face embeddings saved to {embeddings_path}")

Face embeddings saved to C:\Code\vision-attendance-system\face_embeddings.npy
